In [2]:
import gurobipy as grb
import pandas as pd


categories = {
    "good":100,
    "bad":10,
    "terrible":1
}

multipliers = {
    (0,0,0):1,
    (1,0,0):1,
    (0,1,0):10,
    (0,0,1):100,
    (2,0,0):1,
    (1,1,0):10,
    (1,0,1):2500,
    (0,2,0):1000,
    (0,1,1):10000,
    (0,0,2):100000,
    (3,0,0):1,
    (2,1,0):10,
    (2,0,1):1000,
    (1,2,0):500,
    (1,1,1):1000,
    (1,0,2):100000,
    (0,3,0):5000,
    (0,2,1):10000,
    (0,1,2):100000,
    (1,1,2):100000,
    (0,0,3):100000
}

venues_all = {
    220:"good",
    502:"good",
    503:"good",
    504:"good",
    505:"good",
    506:"bad",
    514:"good",
    602:"good",
    606:"bad",
    608:"bad",
    611:"good",
    613:"good",
    622:"bad",
    624:"good"
}


venues = {}
venues[1] = {
    220:100,
    502:100,
    503:100,
    504:100,
    505:100,
    506:1,
    514:100,
}

venues[2] = {
    602:100,
    606:50,
    608:50,
    611:100,
    613:100,
    622:50,
    624:100
}


def allocate(rd):
    current = rd
    past = rd-1
    
    data = pd.read_csv("data_afterr{}.csv".format(past),index_col=0)#.format(past),index_col=0)
    
    teams_records = {}
    teams_multipliers = {}
    
    for ind,row in data.iterrows():
        good = 0
        bad = 0
        terrible = 0
        
        for rd_col in (1,2,3):
            if row[rd_col] == "--":
                pass
            elif row[rd_col] == "nan":
                pass
            elif venues_all[row[rd_col]] == "good":
                good += 1
            elif venues_all[row[rd_col]] == "bad":
                bad += 1
            elif venues_all[row[rd_col]] == "terrible":
                terrible += 1
        teams_records[ind] = [good,bad,terrible]
        
        # Add the highest multiplier to Jonesboro 1/2 and AIS so that they get large rooms regardless
        if ind == "McCollum":
            teams_multipliers[ind] = 100000
        elif ind == "Tucker":
            teams_multipliers[ind] = 100000
        elif ind == "Baldwin":
            teams_multipliers[ind] = 100000
        else:
            teams_multipliers[ind] = multipliers[(good,bad,terrible)]
    
    print(teams_multipliers)
    
    pairings = {}
    pairingsd1_csv = pd.read_csv("pairings_r{}_div1.csv".format(current),index_col="rank")
    pairingsd2_csv = pd.read_csv("pairings_r{}_div2.csv".format(current),index_col="rank")
    
    
    pairings[1] = {}
    pairings[2] = {}
    for ind,row in pairingsd1_csv.iterrows():
        pairings[1][ind] = [row[0],row[1]]
    for ind,row in pairingsd2_csv.iterrows():
        pairings[2][ind] = [row[0],row[1]]

        
    costs = {}
    
    costs[1] = {}
    costs[2] = {}
    
    for pairingkey,pairingvalue in pairings[1].iteritems():
        pairing_cost = 0
        for team in pairingvalue:
            pairing_cost += teams_multipliers[team]
            
        for venue,cost in venues[1].iteritems():
            combined_cost = pairing_cost * cost
            costs[1][pairingkey,venue] = combined_cost
            
    for pairingkey,pairingvalue in pairings[2].iteritems():
        pairing_cost = 0
        for team in pairingvalue:
            pairing_cost += teams_multipliers[team]
            
        for venue,cost in venues[2].iteritems():
            combined_cost = pairing_cost * cost
            costs[2][pairingkey,venue] = combined_cost
        
            
    
    m1 = grb.Model("Division 1 Allocation")
    x1 = {}
    
    for key,value in costs[1].iteritems():
        x1[key] = m1.addVar(vtype = grb.GRB.BINARY, obj = value, name="x1_{}".format(key))
            
    #Each venue gets one pairing
    for key,value in venues[1].iteritems():
        m1.addConstr(sum(x1[pairing,key] for pairing in pairings[1].keys()) == 1)
    #Each pairing gets one venue
    for key,value in pairings[1].iteritems():
        m1.addConstr(sum(x1[key,venue] for venue in venues[1].keys()) == 1)

        
        ## Bye-Buster in 506 
        m1.addConstr(x1[7,506]==1)
    
        
    m1.modelSense = grb.GRB.MAXIMIZE
    
    m1.optimize()    
            
    m2 = grb.Model("Division 2 Allocation")
    x2 = {}
    
    for key,value in costs[2].iteritems():
        x2[key] = m2.addVar(vtype = grb.GRB.BINARY, obj = value, name="x2_{}".format(key))
            
    #Each venue gets one pairing
    for key,value in venues[2].iteritems():
        m2.addConstr(sum(x2[pairing,key] for pairing in pairings[2].keys()) == 1)
    #Each pairing gets one venue
    for key,value in pairings[2].iteritems():
        m2.addConstr(sum(x2[key,venue] for venue in venues[2].keys()) == 1)
        
    m2.modelSense = grb.GRB.MAXIMIZE
    
    m2.optimize()
    
    
    
    print("-----")
    print("DIVISION 1 ALLOCATION - ROUND {}".format(current))
    for key,value in costs[1].iteritems():
        if x1[key].x != 0:
            print("Rank {} ({} v {}) in Room {}").format(key[0],pairings[1][key[0]][0],pairings[1][key[0]][1],key[1])
     
    print("-----")
    

    print("DIVISION 2 ALLOCATION - ROUND {}".format(current))
    for key,value in costs[2].iteritems():
        if x2[key].x != 0:
            print("Rank {} ({} v {}) in Room {}").format(key[0],pairings[2][key[0]][0],pairings[2][key[0]][1],key[1])

In [4]:
allocate(4)
#Put Bye-Buster in 506

{'Santana': 1, 'Grape': 1, 'Young': 1, 'Creedence': 500, 'Bordin': 1, 'Desmond': 10, 'Biafra': 500, 'Greenday': 1, 'Exodus': 5000, 'Nightranger': 500, 'Weir': 10, 'Buckingham': 10, 'Train': 1, 'Franti': 1, 'Metallica': 1, 'Stone': 10, 'McKernan': 10, 'Garcia': 1, 'Journey': 10, 'Testament': 1, 'Joplin': 500, 'Moore': 1, 'Lewis': 1, 'Crows': 10, 'Miller': 500, 'Call': 500, 'Airplane': 10, 'Lesh': 10}
Optimize a model with 21 rows, 49 columns and 105 nonzeros
Variable types: 0 continuous, 49 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5301.0000000
Presolve removed 9 rows and 13 columns
Presolve time: 0.01s
Presolved: 12 rows, 36 columns, 72 nonzeros
Variable types: 0 continuous, 36 integer (36 binary)

Root relaxation: cutoff, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |  